In [2]:
!git clone https://github.com/deepseek-ai/DeepSeek-VL2.git

Cloning into 'DeepSeek-VL2'...
remote: Enumerating objects: 211, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 211 (delta 61), reused 39 (delta 39), pack-reused 129 (from 2)
Receiving objects: 100% (211/211), 31.05 MiB | 5.78 MiB/s, done.
Resolving deltas: 100% (91/91), done.


In [12]:
!curl -O https://bootstrap.pypa.io/get-pip.py
!python get-pip.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2246k  100 2246k    0     0  5324k      0 --:--:-- --:--:-- --:--:-- 5335k
  Using cached pip-25.0.1-py3-none-any.whl.metadata (3.7 kB)
  Using cached setuptools-76.0.0-py3-none-any.whl.metadata (6.7 kB)
  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
Using cached pip-25.0.1-py3-none-any.whl (1.8 MB)
Using cached setuptools-76.0.0-py3-none-any.whl (1.2 MB)
Using cached wheel-0.45.1-py3-none-any.whl (72 kB)


In [3]:
%cd DeepSeek-VL2

/home/quynd/vidCapLLM/DeepSeek-VL2


/home/quynd/myenv/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
!pip install -e .

Obtaining file:///home/quynd/vidCapLLM/DeepSeek-VL2
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for deepseek_vl2 (pyproject.toml) ... done
  Created wheel for deepseek_vl2: filename=deepseek_vl2-1.0.0-0.editable-py3-none-any.whl size=14507 sha256=98d4edddc9777ac25700aa4e3e61b171f34d961f06e001716233e0e3d5695f1b
  Stored in directory: /tmp/pip-ephem-wheel-cache-ime5ddhk/wheels/c7/3a/be/2df7ced65a0650263c4048d2af8f4963bf95998ca4e8f893a2
Successfully built deepseek_vl2
  Attempting uninstall: deepseek_vl2
    Found existing installation: deepseek_vl2 1.0.0
    Uninstalling deepseek_vl2-1.0.0:
      Successfully uninstalled deepseek_vl2-1.0.0


In [5]:
import torch
from transformers import AutoModelForCausalLM

from deepseek_vl2.models import DeepseekVLV2Processor, DeepseekVLV2ForCausalLM
from deepseek_vl2.utils.io import load_pil_images


# specify the path to the model
model_path = "deepseek-ai/deepseek-vl2-tiny"
vl_chat_processor: DeepseekVLV2Processor = DeepseekVLV2Processor.from_pretrained(model_path)
tokenizer = vl_chat_processor.tokenizer

vl_gpt: DeepseekVLV2ForCausalLM = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)
vl_gpt = vl_gpt.to(torch.bfloat16).cuda().eval()

## single image conversation example
## Please note that <|ref|> and <|/ref|> are designed specifically for the object localization feature. These special tokens are not required for normal conversations.
## If you would like to experience the grounded captioning functionality (responses that include both object localization and reasoning), you need to add the special token <|grounding|> at the beginning of the prompt. Examples could be found in Figure 9 of our paper.
conversation = [
    {
        "role": "<|User|>",
        "content": "<image>\n<|ref|>The giraffe at the back.<|/ref|>.",
        "images": ["./images/visual_grounding_1.jpeg"],
    },
    {"role": "<|Assistant|>", "content": ""},
]

# load images and prepare for inputs
pil_images = load_pil_images(conversation)
prepare_inputs = vl_chat_processor(
    conversations=conversation,
    images=pil_images,
    force_batchify=True,
    system_prompt=""
).to(vl_gpt.device)

# run image encoder to get the image embeddings
inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

# run the model to get the response
outputs = vl_gpt.language.generate(
    inputs_embeds=inputs_embeds,
    attention_mask=prepare_inputs.attention_mask,
    pad_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=512,
    do_sample=False,
    use_cache=True
)

answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=False)
print(f"{prepare_inputs['sft_format'][0]}", answer)

ModuleNotFoundError: No module named 'torch'